In [ ]:
import pandas as pd
import json
import datetime as dt
from  geopy.geocoders import Nominatim
import meteomatics.api as weather
import seaborn as sns
import matplotlib.pyplot as plt
from api_config import *

In [ ]:
df_O3 = pd.read_csv("datasets/O3 Summary.csv")
df_NO2 = pd.read_csv("datasets/NO2 Summary.csv")

In [ ]:
df_O3 = df_O3.loc[(df_O3["O3 Number Density (cm^-3)"] > 0 )]
df_NO2 = df_NO2.loc[(df_NO2["NO2 Number Density (cm^-3)"] > 0 )]

In [ ]:
import numpy as np
df_O3['log_O3'] = np.log2(df_O3['O3 Number Density (cm^-3)'])
df_NO2['log_NO2'] = np.log2(df_NO2['NO2 Number Density (cm^-3)'])


In [ ]:
df_O3_sample = df_O3.sample(n=1000) 
df_NO2_sample = df_NO2.sample(n=1000) 

In [ ]:
df_O3_sample.hist(column="log_O3", bins=int(5))
df_NO2_sample.hist(column="log_NO2", bins=int(5))

In [ ]:
# df_O3_sample 
df_NO2_sample

In [ ]:
for i, row in df_NO2_sample.iterrows():
    dt = pd.to_datetime(row["Time (TAI 1993)"])
    dt = str(dt.isoformat()) + "Z"
    alt = row[" Altitude (km)"]
    lat = row["Latitude (degree)"]
    lng = row["Longitude (degree)"]
    no2 = row["log_NO2"]
    print(dt, alt,lat,lng,no2)
    # tmp = get_data()
    if i > 1:
        break

In [ ]:
def get_data(lat, lng, dt):
    url=f"https://api.meteomatics.com/{dt}/t_1000m:F,t_5m:F/{lat},{lng}/json?model=mix"
    df = weather.query_api(url=url, username=USER, password=PASSWORD)
    response = dict(df.json())
    return response

In [ ]:
r = get_data(33.0455, -105.431, "1970-01-01T00:00:00.579058Z")
r

In [ ]:
data = response["data"][0]["coordinates"][0]["dates"]

In [ ]:
df = pd.DataFrame(data)
df

In [ ]:
fig = px.line(df, x='date', y="value")
fig

In [ ]:
start = str(dt.datetime.utcnow().isoformat()) + "Z"
range = (df['date'] > start)
df1 = df.loc[range]
df1

In [ ]:
max_hs = df1["value"].max()

In [ ]:
from scipy import stats

df.date = pd.to_datetime(df.date)
df['date_ordinal'] = pd.to_datetime(df['date']).map(dt.datetime.toordinal)
slope, intercept, r_value, p_value, std_err = stats.linregress(df['date_ordinal'], df['value'])
print(slope,
intercept,
r_value,p_value,
std_err)